In [39]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.schema import HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

import json
from langchain.docstore.document import Document


# Data Preparation

## Import scraped data from scraper

In [40]:
# from scraper import json_list

In [41]:
# json_list[:5]

In [42]:
# # เซฟทั้งหมดเป็นไฟล์เดียว
# with open("data.json", "w", encoding="utf-8") as f:
#     json.dump(json_list, f, ensure_ascii=False, indent=2)


## อ่านข้อมูลจาก json มาเก็บในลิสต์

In [43]:
json_list_x = []
with open("data.json", "r", encoding="utf-8") as f:
    json_list_x = json.load(f)


## Change json_list to Document

In [44]:
documents = []
for item in json_list_x:
    content = f"subject: {item['subject']}\n gender: {item['gender']}\n age: {item['age']}\n symptoms: {', '.join(item['symptoms'])}\n question: {item['question']}\n answer: {item['answer']}"
    documents.append(Document(page_content=content, metadata={"source": "scraper"}))



## Embeding and Store to VectorDB

In [45]:
embedder = OllamaEmbeddings(model="bge-m3")


In [46]:
# vector_store = FAISS.from_documents(documents, embedder)

vector_store = FAISS.load_local("D:/faiss_storage", 
                                embedder,
                                allow_dangerous_deserialization=True
)
print("Loaded vector store from disk")

Loaded vector store from disk


In [47]:
retriever = vector_store.as_retriever()



# Create chat

In [48]:
llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",
                    temperature=0.4)


In [49]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "คุณคือผู้แพทย์เชี่ยวชาญด้านสุขภาพที่มีความรู้ลึกซึ้งทั้งจากประสบการณ์และฐานข้อมูลทางการแพทย์ "
     "คุณจะต้องให้คำแนะนำที่แม่นยำและปลอดภัย โดยเริ่มต้นด้วยประโยค 'คำตอบจากแพทย์ผู้เชี่ยวชาญ' "
     "จากนั้นให้คำตอบแบ่งเป็นหัวข้อ: สาเหตุที่เป็นไปได้, แนวทางการดูแลเบื้องต้น, คำแนะนำเพิ่มเติม "
     " ใส่ข้อความ 'คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ \nhttps://bit.ly/doctor-refer-agnos' ในช่วงท้ายของการสนทนา"),
    
    ("human", 
     "มีผู้ป่วยที่มีข้อมูลดังนี้:\n"
     "- หัวข้อ: {subject}\n"
     "- เพศ: {gender}\n"
     "- อายุ: {age} ปี\n"
     "- อาการ: {symptoms}\n"
     "- คำถาม: {question}\n"
     "ข้อมูลจากฐานข้อมูลที่เกี่ยวข้อง:\n{context}\n"
     "กรุณาวิเคราะห์และให้คำตอบอย่างละเอียด โดยอ้างอิงจากทั้งความรู้ของคุณและข้อมูลที่ให้มาในส่วนของ context อย่างรอบคอบ"
     "หากข้อมูลใน context ไม่เพียงพอ ให้ใช้ความรู้ทั่วไปของคุณในการตอบคำถามอย่างระมัดระวังและแนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญ"
     )
])

In [50]:
from langchain.schema.runnable import RunnableLambda

def fill_missing_fields(inputs):
    return {
        "subject": inputs.get("subject", "ไม่ระบุ"),
        "gender": inputs.get("gender", "ไม่ระบุ"),
        "age": inputs.get("age", "ไม่ระบุ"),
        "symptoms": inputs.get("symptoms", "ไม่ระบุ"),
        "question": inputs.get("question", "ไม่ระบุ"),
        "context": inputs.get("context", "ไม่มีข้อมูลจากฐานข้อมูล")
    }

input_preprocessor = RunnableLambda(fill_missing_fields)


In [51]:
rag_chain = (
    {
        "context": retriever,
        "subject": RunnablePassthrough(),
        "gender": RunnablePassthrough(),
        "age": RunnablePassthrough(),
        "symptoms": RunnablePassthrough(),
        "question": RunnablePassthrough(),

    }
    | input_preprocessor
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
data_input =[ {
    "subject": "อาการปวดหัวเรื้อรัง",
    "gender": "หญิง",
    "age": "35",
    "symptoms": "ปวดหัวบริเวณขมับทั้งสองข้างมาเกือบทุกวัน โดยเฉพาะช่วงบ่าย รู้สึกตึงๆ และมีอาการคลื่นไส้ร่วมด้วย",
    "question": "อาการแบบนี้เกิดจากอะไร และควรดูแลตัวเองอย่างไร?"
},
{
    "subject": "การนอนหลับในผู้สูงอายุ",
    "gender": "ชาย",
    "age": "68",
    "symptoms": "นอนหลับไม่สนิท ตื่นบ่อยตอนกลางคืน และรู้สึกไม่สดชื่นตอนเช้า",
    "question": "ควรปรับพฤติกรรมหรือมีวิธีดูแลสุขภาพอย่างไรให้หลับดีขึ้น?"
},
{
    "subject": "ภาวะขาดวิตามินบี 12 (Vitamin B12 deficiency)",
    "gender": "หญิง",
    "age": "29",
    "symptoms": [
      "เหนื่อยง่าย",
      "มือเท้าชา",
      "เวียนหัว",
      "ลืมง่าย",
      "อ่อนแรง"
    ],
    "question": "ช่วงนี้รู้สึกเหนื่อยง่ายมากค่ะ เดินนิดเดียวก็หอบ มือเท้าชาบ่อย ๆ โดยเฉพาะตอนตื่นนอน รู้สึกเวียนหัวและลืมง่ายขึ้นเรื่อย ๆ อยากทราบว่าอาการแบบนี้เกิดจากอะไร และควรตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคกรดไหลย้อน (GERD)",
    "gender": "ชาย",
    "age": "34",
    "symptoms": [
      "แสบร้อนกลางอก",
      "เรอเปรี้ยว",
      "ไอเรื้อรัง",
      "แน่นหน้าอก",
      "กลืนลำบาก"
    ],
    "question": "ผมมีอาการแสบร้อนกลางอกหลังทานอาหาร โดยเฉพาะอาหารเผ็ดหรือมัน ๆ เรอเปรี้ยวบ่อยมาก และบางครั้งรู้สึกแน่นหน้าอกจนหายใจไม่สะดวก อยากทราบว่าเป็นกรดไหลย้อนหรือเปล่า และควรปรับพฤติกรรมยังไงครับ?"
  },
  {
    "subject": "ภาวะถุงน้ำรังไข่หลายใบ (PCOS)",
    "gender": "หญิง",
    "age": "26",
    "symptoms": [
      "ประจำเดือนมาไม่ปกติ",
      "สิวขึ้นเยอะ",
      "ขนดก",
      "น้ำหนักขึ้นง่าย",
      "ปวดท้องน้อย"
    ],
    "question": "ประจำเดือนมาไม่ตรงเวลาเลยค่ะ บางเดือนก็ไม่มาเลย สิวขึ้นเยอะมากทั้งที่ดูแลผิวดีแล้ว ขนตามตัวก็ดูเยอะขึ้น น้ำหนักขึ้นง่ายมากทั้งที่กินเท่าเดิม อยากทราบว่าอาการแบบนี้เกี่ยวกับฮอร์โมนหรือเปล่า และควรไปตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคภูมิแพ้อากาศ (Allergic Rhinitis)",
    "gender": "ชาย",
    "age": "18",
    "symptoms": [
      "จามบ่อย",
      "คัดจมูก",
      "น้ำมูกไหล",
      "คันจมูก",
      "ตาแดง"
    ],
    "question": "ผมมีอาการจามบ่อยมาก โดยเฉพาะตอนเช้า ๆ หรือเวลาอยู่ในห้องแอร์ น้ำมูกไหล คัดจมูก และบางครั้งตาแดงคันร่วมด้วย อยากทราบว่าเป็นภูมิแพ้อากาศหรือเปล่า และมีวิธีดูแลตัวเองยังไงบ้างครับ?"
  }
]

In [53]:
# for i, data in enumerate(data_input):
#     result = rag_chain.invoke(data)
#     print(f"หัวข้อที่ {i+1}\n")
#     print(result)




In [54]:
my_question = rag_chain.invoke({    
    "subject": "กรดไหลย้อน อาหารไม่ย่อย",
    "gender": "ชาย",
    "age": "24",
    "symptoms": "แสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอ",
    "question": "มีคืนนึงตอนผมกำลังจะนอนจู่ๆผมก็รู้สึกแสบคอขึ้นมา แล้วพอตื่นมาก็รู้สึกว่าเหมือนมีก้อนอะไรมาจุกที่คอ ผมเลยค้นหาจากอินเตอร์เน็ตก็พบว่าผมเป็นกรดไหลย้อนจึงไปซื้อยาที่ร้านขายยามากินเป็นเวลา 1 อาทิตย์ จนพอผมเริ่มรู้สึกดีขึ้นผมจึงไปกินบุฟเฟ่ต์ชาบูแล้วทีนี้ผมรู้สึกว่าอาหารมันไม่ย่อยเลยแม้แต่ตอนกำลังจะนอนก็รู้สึกได้ว่ามีอาหารอยู่ในท้องจนถึงวัดถัดไป ควรรักษาต่อยังไงดีครับ"
})

print(my_question)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

สาเหตุที่เป็นไปได้:
อาการแสบคอ แสบท้อง ท้องอืด และจุกคอที่ผู้ป่วยกล่าวถึง อาจเกิดจากกรดไหลย้อน (GERD) ซึ่งเป็นภาวะที่กรดในกระเพาะอาหารไหลย้อนขึ้นไปในหลอดอาหาร ทำให้เกิดอาการแสบคอ แสบท้อง และจุกแน่นท้องได้

แนวทางการดูแลเบื้องต้น:
1. หลีกเลี่ยงการกินอาหารที่มีกรดสูง เช่น น้ำอัดลม กาแฟ และอาหารเผ็ด
2. รับประทานอาหารในปริมาณที่พอเหมาะ ไม่ควรทานมากเกินไป
3. งดการนอนหลังรับประทานอาหาร 2-3 ชั่วโมง
4. ปรึกษาแพทย์เกี่ยวกับการใช้ยาลดกรด เช่น omeprazole หรือ ranitidine

คำแนะนำเพิ่มเติม:
หากอาการยังคงอยู่หรือแย่ลง ควรไปพบแพทย์เพื่อทำการตรวจวินิจฉัยและรับคำแนะนำที่เหมาะสม นอกจากนี้ หากมีอาการเจ็บหน้าอก ปวดท้อง หรือใจสั่นร่วมด้วย อาจต้องพิจารณาโรคอื่นๆ ที่อาจเป็นสาเหตุได้ เช่น โรคหัวใจหรือกระเพาะอาหารอักเสบ

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [55]:
my_question2 = rag_chain.invoke({    
    "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
})

print(my_question2)

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

สาเหตุที่เป็นไปได้: อาการนอนไม่หลับอาจเกิดจากความเครียด ความวิตกกังวล หรือปัญหาทางจิตใจ เช่น โรคซึมเศร้าหรือภาวะหมดไฟในการทำงาน (burnout syndrome) นอกจากนี้ยังอาจเกี่ยวข้องกับการเปลี่ยนแปลงในชีวิตประจำวัน เช่น การเปลี่ยนงาน การหย่าร้าง หรือการสูญเสียคนที่รัก

แนวทางการดูแลเบื้องต้น: ลองปรับพฤติกรรมการนอนตามคำแนะนำที่ได้กล่าวไปข้างต้น เช่น จัดห้องนอนให้มีบรรยากาศเงียบสงบ ไม่มีแสงเสียงรบกวน ใช้ห้องนอนเพื่อการนอนเท่านั้น และพยายามไม่คิดเรื่องเครียดหรือกังวลในเวลานอน นอกจากนี้ควรมีการออกกำลังกายอย่างสม่ำเสมอ รับประทานอาหารที่มีประโยชน์ และหากิจกรรมผ่อนคลายทำเพื่อช่วยลดความเครียด

คำแนะนำเพิ่มเติม: หากอาการนอนไม่หลับยังคงมีอยู่ หรือมีอาการอื่นๆ ที่เกี่ยวข้อง เช่น เศร้า หดหู่ วิตกกังวล ควรไปพบแพทย์หรือจิตแพทย์เพื่อรับการประเมินและรักษาอย่างเหมาะสม

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [56]:
my_question3 = rag_chain.invoke({    
    "subject": "เข่าดังก๊อกแก๊กเวลาเดิน",
    "symptoms": "เข่าดังก๊อกแก๊กเวลาเดิน, ปวดเข่า",
    })
print(my_question3)

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

สาเหตุที่เป็นไปได้:
อาการที่เกิดขึ้น เช่น เข่าดังก๊อกแก๊กเวลาเดินและปวดเข่า อาจเกิดจากหลายสาเหตุ ได้แก่ การบาดเจ็บของกล้ามเนื้อหรือกระดูก เอ็นอักเสบ หรือโรคข้อเข่าอักเสบ (Arthritis) ซึ่งอาจเป็นสาเหตุที่ทำให้เกิดเสียงและความเจ็บปวดในขณะเดิน

แนวทางการดูแลเบื้องต้น:
สำหรับการดูแลอาการที่เกิดขึ้น ควรเริ่มจากการพักการใช้งานเข่า เช่น หลีกเลี่ยงการเดินขึ้นลงบันไดหรือการกระโดด และใช้การประคบร้อนหรือเย็นเพื่อบรรเทาอาการปวด นอกจากนี้ควรทานยาแก้อักเสบที่ไม่ใช่สเตียรอยด์ (NSAIDs) เช่น diclofenac เพื่อลดอาการปวดและบวม หากอาการยังไม่ดีขึ้นหรือแย่ลง ควรไปพบแพทย์เพื่อทำการตรวจเพิ่มเติม

คำแนะนำเพิ่มเติม:
หากผู้ป่วยมีอาการที่ไม่ดีขึ้นหรือมีอาการอื่นๆ ร่วมด้วย เช่น อาการบวม แดง หรือร้อนบริเวณเข่า ควรไปพบแพทย์เฉพาะทางด้านกระดูกและข้อ เพื่อให้ได้รับการวินิจฉัยและการรักษาที่ถูกต้อง

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [57]:
my_question4 = rag_chain.invoke({    
    "question": "รู้สึกเดินๆอยู่แล้วเข่าอ่อนบ่อยๆ ไม่ก็ข้อเท้าเกือบจะพลิกบ่อยๆ มันคืออาการอะไรครับ"
})
print(my_question4)

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

หัวข้อ: อาการเข่าอ่อนและข้อเท้าเกือบพลิก

สาเหตุที่เป็นไปได้:
อาการเข่าอ่อนและข้อเท้าเกือบพลิกอาจเกิดจากหลายสาเหตุ เช่น การบาดเจ็บที่ข้อต่อหรือกล้ามเนื้อรอบๆ, โรคข้ออักเสบ, หรือปัญหาทางระบบประสาท

แนวทางการดูแลเบื้องต้น:
1. พักการใช้งานของข้อต่อที่มีอาการ เช่น ลดการเดินหรือวิ่ง
2. ใช้การประคบร้อนหรือเย็นเพื่อบรรเทาอาการเจ็บปวด
3. ยกขาสูงเมื่ออยู่ในท่านั่งหรือนอน เพื่อลดการบวมและความดันในข้อต่อ
4. รับประทานยาแก้ปวดตามที่แพทย์สั่ง

คำแนะนำเพิ่มเติม:
หากอาการไม่ดีขึ้นภายใน 1-2 สัปดาห์ หรือมีอาการรุนแรง เช่น ข้อต่อผิดรูปหรือบวมมาก ควรไปพบแพทย์เพื่อทำการตรวจวินิจฉัยและรักษาอย่างเหมาะสม

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos
